In [0]:
import pickle
import tensorflow as tf
import librosa
import numpy as np
import itertools
from itertools import combinations
import random

In [0]:
with open('hw4_trs.pkl', 'rb') as f:
    trs = pickle.load(f)

In [3]:
print(trs.shape)

(500, 16180)


In [0]:
padded_trs = np.zeros((500,22631))
padded_trs[:, :16180] = trs

In [5]:
print(padded_trs.shape)

(500, 22631)


In [0]:
with open('hw4_tes.pkl', 'rb') as f:
    tes = pickle.load(f)

In [7]:
print(tes.shape)

(200, 22631)


In [8]:
train_data=[]

for i in range(500):
    S=librosa.stft(padded_trs[i], n_fft=1024, hop_length=512)
    train_data.append(S)

train_data=np.asarray(train_data)
print(train_data.shape)

(500, 513, 45)


In [9]:
train_data=np.abs(train_data).transpose((0,2,1))
print(train_data.shape)

(500, 45, 513)


In [10]:
test_data=[]
for i in range(200):
    S=librosa.stft(tes[i], n_fft=1024, hop_length=512)
    test_data.append(S)

test_data=np.asarray(test_data)
print(test_data.shape)

(200, 513, 45)


In [11]:
test_data=np.abs(test_data).transpose((0,2,1))
print(test_data.shape)

(200, 45, 513)


In [0]:
def generate_pairs(data):
  l = len(data)
  all_pos_pairs = list()
  all_neg_pairs = list()
  
  for i in range(0,l,10):
    pos_pair = list(combinations(data[i:i+10],2))
    pos_labels = [1 for i in range(len(pos_pair))]
    all_pos_pairs += pos_pair
    
    
    temp_neg_pair = list(itertools.product(data[i:i+10], list(itertools.chain(data[i+10:l] , data[0:i]))))
    neg_pair = random.sample(temp_neg_pair, 45)
    all_neg_pairs += neg_pair
  
  pos_labels = [1 for i in range(len(all_pos_pairs))]
  neg_labels = [0 for i in range(len(all_neg_pairs))]
  combined_pairs = all_pos_pairs + all_neg_pairs
  combined_labels = pos_labels + neg_labels
  
  temp = list(zip(combined_pairs, combined_labels))
  random.shuffle(temp)
  combined_pairs, combined_labels = zip(*temp)
  return np.asarray(combined_pairs), np.asarray(combined_labels)

In [13]:
pos_pair = np.asarray(list(combinations(train_data[0:0+10],2)))
print(pos_pair.shape)

neg_pair = list(itertools.product(train_data[0:0+10], train_data[10:500]))
neg_pair = np.asarray(random.sample(neg_pair, 45))
print(neg_pair.shape)

(45, 2, 45, 513)
(45, 2, 45, 513)


In [14]:
train_set, train_labels = generate_pairs(train_data)

print(train_set.shape)
print(train_labels.shape)


(4500, 2, 45, 513)
(4500,)


In [15]:
left_train_set = train_set[:,0,:,:]
print(left_train_set.shape)

(4500, 45, 513)


In [16]:
right_train_set = train_set[:,1,:,:]
print(right_train_set.shape)

(4500, 45, 513)


In [17]:
test_set, test_labels = generate_pairs(test_data)

print(test_set.shape)
print(test_labels.shape)

(1800, 2, 45, 513)
(1800,)


In [18]:
left_test_set = test_set[:,0,:,:]
print(left_test_set.shape)

(1800, 45, 513)


In [19]:
right_test_set = test_set[:,1,:,:]
print(right_test_set.shape)

(1800, 45, 513)


In [0]:
X_left = tf.placeholder("float", [None,45, 513])
X_right = tf.placeholder("float", [None,45, 513])

Y = tf.placeholder("float", [None])

n_hidden_1 = 512
n_hidden_2 = 512
batch_size = 100

In [0]:

def construct_model(X, reuse):
    
    GRU_stack = []
    conv_layer_1 = tf.layers.conv1d(inputs=X, filters=32, kernel_size=2, activation = tf.nn.relu, reuse = reuse)
    max_pool_1 = tf.layers.max_pooling1d(inputs=conv_layer_1, pool_size=2, strides=2)
    
    
    gru_cell_1 = tf.nn.rnn_cell.GRUCell(n_hidden_1, kernel_initializer=tf.contrib.layers.xavier_initializer(), reuse = reuse)   
    GRU_stack.append(gru_cell_1)
    
    gru_cell_2 = tf.nn.rnn_cell.GRUCell(n_hidden_2, kernel_initializer=tf.contrib.layers.xavier_initializer(), reuse = reuse)
    GRU_stack.append(gru_cell_2)
    
    stacked_cells = tf.nn.rnn_cell.MultiRNNCell(GRU_stack)
    
    gru_op, _ = tf.nn.dynamic_rnn(stacked_cells, max_pool_1, dtype = tf.float32, swap_memory = True)
    norm_out = tf.contrib.layers.batch_norm(gru_op, is_training=True, updates_collections=None)
    
    drop_out_1 = tf.nn.dropout(norm_out, 0.5)
    fc1 = tf.layers.flatten(drop_out_1)

    output_layer = tf.layers.dense(inputs = fc1, 
                           units = 513, 
                           kernel_initializer = tf.contrib.layers.variance_scaling_initializer(),
                           activation = tf.nn.tanh,
                           bias_initializer = tf.zeros_initializer(),
                           reuse=reuse)
#     print(output_layer.get_shape())

    return output_layer

In [22]:
output_left = construct_model(X_left, reuse=False)
output_right = construct_model(X_right, reuse=True)

Instructions for updating:
Use keras.layers.conv1d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling1d instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Us

In [23]:
dotProduct = tf.reduce_sum(tf.multiply(output_left,output_right),axis = 1 ,name = 'dotproduct')
print(dotProduct.get_shape())

yPred = tf.nn.sigmoid(dotProduct)
print(yPred.get_shape())

binarisedOutput = tf.cast(tf.math.greater(yPred,0.5), tf.int16)
print(binarisedOutput.get_shape())


accuracy = tf.metrics.accuracy(labels = Y, predictions = binarisedOutput)

lossCalcu = tf.nn.sigmoid_cross_entropy_with_logits(labels = Y, logits = dotProduct)
lossCalcu  = tf.reduce_sum(lossCalcu)
step = tf.Variable(0, trainable=False)
rate = tf.train.exponential_decay(0.001, step, 1, 0.99)
gradOptimizer = tf.train.AdamOptimizer(learning_rate=rate)

train = gradOptimizer.minimize(lossCalcu)
# accuracy = RSquared(LastOutput,Y)

initialise = tf.global_variables_initializer()

(?,)
(?,)
(?,)
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [24]:
sess = tf.InteractiveSession()
sess.run(initialise)
for epoch in range(100):
  loss = []
  for i in range(0, int(len(train_set)/batch_size)):
    r = random.randint(0, len(train_set)-batch_size)

#     xbatchLeft = left_train_set[i*batch_size:(i+1)*batch_size,:,:]
#     xbatchRight = right_train_set[i*batch_size:(i+1)*batch_size,:,:]
#     yBatch = train_labels[i*batch_size:(i+1)*batch_size]
    
    xbatchLeft = left_train_set[r:r+batch_size,:,:]
    xbatchRight = right_train_set[r:r+batch_size,:,:]
    yBatch = train_labels[r:r+batch_size]
    
    sess.run(train,feed_dict ={X_left: xbatchLeft,
                               X_right: xbatchRight,Y:yBatch,})
    loss += sess.run([lossCalcu], feed_dict ={X_left: xbatchLeft,
                                              X_right: xbatchRight,Y:yBatch})  

  print("Step " + str(epoch) + ", Loss= " + str(sum(loss)))
  predictedytest = binarisedOutput.eval(feed_dict ={X_left: left_test_set,
                                      X_right: right_test_set,Y:test_labels})
  totalequal = sum(predictedytest==test_labels) 
  accuracy =totalequal/ test_labels.shape
  accuracy =accuracy*100
  print("Accuracy on test set = ", accuracy)

Step 0, Loss= 139144.17260742188
Accuracy on test set =  [66.77777778]
Step 1, Loss= 86360.95776367188
Accuracy on test set =  [66.33333333]
Step 2, Loss= 68927.22592163086
Accuracy on test set =  [62.55555556]
Step 3, Loss= 67467.93090820312
Accuracy on test set =  [66.11111111]
Step 4, Loss= 54504.642150878906
Accuracy on test set =  [67.83333333]
Step 5, Loss= 39631.10614776611
Accuracy on test set =  [65.55555556]
Step 6, Loss= 36688.14434814453
Accuracy on test set =  [66.05555556]
Step 7, Loss= 31728.97315979004
Accuracy on test set =  [66.05555556]
Step 8, Loss= 25309.4853515625
Accuracy on test set =  [66.88888889]
Step 9, Loss= 25825.53832244873
Accuracy on test set =  [65.33333333]
Step 10, Loss= 16869.779090881348
Accuracy on test set =  [66.44444444]
Step 11, Loss= 13999.753005981445
Accuracy on test set =  [68.27777778]
Step 12, Loss= 12924.104054450989
Accuracy on test set =  [67.11111111]
Step 13, Loss= 14278.611179351807
Accuracy on test set =  [66.33333333]
Step 14, Lo

In [0]:
predictedytest = binarisedOutput.eval(feed_dict ={X_left: left_test_set,
                                      X_right: right_test_set,Y:test_labels})

In [26]:
totalequal = sum(predictedytest==test_labels) 
accuracy =totalequal/ test_labels.shape
accuracy =accuracy*100
print(accuracy)

[74.55555556]
